In [1]:
import pandas as pd
import numpy as np
import time
import re

from bs4 import BeautifulSoup
from datetime import datetime
from tqdm import tqdm
import requests

path = "/Users/Goo/Boaz/3차 미니프로젝트/Data/"

In [9]:
# 필요한 키워드 입력
search_content = input("검색할 키워드를 입력해주세요: ")
max_news = int(input("\n몇 개의 뉴스를 크롤링할지 입력해주세요. ex) 1000(숫자만입력): "))

검색할 키워드를 입력해주세요: 금산인삼축제

몇 개의 뉴스를 크롤링할지 입력해주세요. ex) 1000(숫자만입력): 200


In [10]:
# 크롤링할 기간 설정
# ex) 2023년 금산인삼축제 기간: 2023.10.03 ~ 2023.10.13
# 따라서 축제 시작 전 3개월간의 뉴스를 크롤링
startday = ["2023.07.13"]
endday = ["2023.10.13"]

In [11]:
print("검색 항목 : ", search_content)
print("뉴스 최대 개수 : ", max_news)

검색 항목 :  금산인삼축제
뉴스 최대 개수 :  200


In [12]:
# URL crawling 함수
# -> Redirect 되지않는 네이버뉴스 max_news개가 추출될때까지 크롤링을 계속하는 함수

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

def url_crawling(search_content, start_season, end_season, max_news):
    
    # 집합 형태로 저장해 중복 url 제거
    url_set = set()
    for start_day, end_day in zip(start_season, end_season):    
        for page in tqdm(range(1, 2000, 10)):
            response = requests.get(f"https://search.naver.com/search.naver?where=news&sm=tab_pge&query={search_content}&start={page}&pd=3&ds={start_day}&de={end_day}", headers=headers)
            # page를 넘기다 page가 없으면 종료
            # 200은 HTTP 상태코드중 하나로 OK의 의미를 가짐. 요청이 성공적으로 처리되었음을 나타냄. 200이 아니라는것은 페이지가 없어 페이지를 넘길 수 없다는 의미
            if response.status_code != 200:
                print(f"페이지 {page//10}가 없습니다. Exiting.")
                break
            html = response.text
            soup = BeautifulSoup(html, 'html.parser')
            ul = soup.select_one("div.group_news > ul.list_news")

            if ul is None:
                break
            li_list = ul.find_all('li')
            for li in li_list:
                a_list = li.select('div.news_area > div.news_info > div.info_group > a.info')
                for a_tag in a_list:
                    href = a_tag.get('href')
                    # href 속성값이 "n.news.naver.com"(네이버 뉴스)을 포함하는지 확인한다.
                    if "n.news.naver.com" in href:
                        try:
                            # request.head()로 추출한 url이 rediret되는지 확인한다. redirect 되지않은 url만 저장한다.
                            response = requests.head(href, allow_redirects=True)
                            if response.status_code == 200:
                                url_set.add(href)
                                # 원하는 개수의 기사가 모두 크롤링 되었으면 크롤링 종료
                                if len(url_set) >= max_news:
                                    return list(url_set)
                        except Exception as e:
                            print(f"An error occurred: {e}")
            time.sleep(1)

    return list(url_set)

In [13]:
url = url_crawling(search_content, startday, endday, max_news)
len(url)

 52%|█████████████████████                    | 103/200 [02:24<02:16,  1.41s/it]


140

In [14]:
news_url = url

In [15]:
# 신문사, 제목, 본문 추출
news_company = []
news_title = []
news_content = []

for url in tqdm(news_url):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    company = soup.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_top > a > img[alt]")
    news_company.append(company['alt'] if company else 'None')
    title = soup.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    news_title.append(title.text if title else 'None')
    content = soup.select_one("article#dic_area")
    news_content.append(content.text if content else 'None')

100%|█████████████████████████████████████████| 140/140 [00:21<00:00,  6.55it/s]


In [16]:
# 데이터프레임 생성
columns = ["company", "url", "title", "content"]

data = {
    "company": news_company,
    "url": news_url,
    "title": news_title,
    "content": news_content
}

df_news = pd.DataFrame(data, columns=columns)
df_news.head()

,company,url,title,content
0,아시아경제,https://n.news.naver.com/mnews/article/277/000...,백종원 지역축제 먹거리 사업 개선한다,\n문체부·관광공사와 '착한가격' 먹거리 장 구축내년에 '관광 서비스 품질개선 캠페...
1,경향신문,https://n.news.naver.com/mnews/article/032/000...,"신나게 춤추고, 인삼 실컷 먹고…충남지역 ‘축제’가 쏟아진다",\n\n\n\n\n2022년 열린 금산인삼축제 장면. 금산군 제공‘축제의 계절’이 ...
2,대전일보,https://n.news.naver.com/mnews/article/656/000...,"[줌인] 전어에서 꽃게, 젓갈, 고구마, 빵, 인삼까지… 뭐 먹을까?",\n오슈! 보슈! 드슈! 노슈! 충청권 가을 축제\n\n\n\n올해 대백제전은 9월...
3,뉴스1,https://n.news.naver.com/mnews/article/421/000...,금산세계인삼축제 성공 다짐 자원봉사자 결의대회 개최,\n급수·행사지원·안내·체험·교통 등 5개 분야 총 2060여 명 참여\n\n\n\...
4,KBS,https://n.news.naver.com/mnews/article/056/001...,계룡군문화축제·금산인삼축제 오늘 개막,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n[KBS 대전]\n


In [17]:
len(df_news)

140

In [18]:
# 내용 기준 중복 뉴스 제거
df_news = df_news.drop_duplicates(subset=['content'], keep='first')

In [19]:
len(df_news)

136

In [20]:
# Escape문자 제거
df_news['content'] = df_news['content'].apply(lambda x: re.sub(r'\s+', ' ', x))

In [21]:
df_news.head()

,company,url,title,content
0,아시아경제,https://n.news.naver.com/mnews/article/277/000...,백종원 지역축제 먹거리 사업 개선한다,문체부·관광공사와 '착한가격' 먹거리 장 구축내년에 '관광 서비스 품질개선 캠페인...
1,경향신문,https://n.news.naver.com/mnews/article/032/000...,"신나게 춤추고, 인삼 실컷 먹고…충남지역 ‘축제’가 쏟아진다",2022년 열린 금산인삼축제 장면. 금산군 제공‘축제의 계절’이 왔다. 충남 곳곳...
2,대전일보,https://n.news.naver.com/mnews/article/656/000...,"[줌인] 전어에서 꽃게, 젓갈, 고구마, 빵, 인삼까지… 뭐 먹을까?",오슈! 보슈! 드슈! 노슈! 충청권 가을 축제 올해 대백제전은 9월 23일부터 1...
3,뉴스1,https://n.news.naver.com/mnews/article/421/000...,금산세계인삼축제 성공 다짐 자원봉사자 결의대회 개최,급수·행사지원·안내·체험·교통 등 5개 분야 총 2060여 명 참여 제41회 금산...
4,KBS,https://n.news.naver.com/mnews/article/056/001...,계룡군문화축제·금산인삼축제 오늘 개막,[KBS 대전]


In [27]:
# df_news.to_csv(path + f"{search_content}_{startday[0][:4]}_news.csv", index=False)